In [ ]:
import pandas as pd

In [2]:
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
#from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

from src.bubble import *

device = 'cuda'

Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Darty_trustpilot : 1707329196900x870734705097005300


In [4]:
#df = get("Feedback")
csv_path = 'data/Trustpilot/www.darty.com.xlsx'
#csv_path = 'data/Trustpilot/www.amazon.com.xlsx'
feedbacks_df = pd.read_excel(csv_path, )

In [6]:
feedbacks_df.head()

,Title,Date,Content,Page,Username,Total reviews,Location,Rating
0,Achat Magic Keyboard avec Touch ID et pave numerique chez Darty,"February 07, 2024","Toujours un très bon service de Darty. La récupération du produit a été rapide en magasin. Le personnel est très accueillant et sympathique.Achat d'un clavier Magic Keyboard avec Touch ID et pave numerique : le clavier est de très bonne qualité. La touche Touch ID permet de s'authentifier rapidement et le pavé numérique est très pratique lorsque l'on programme ou réalise des tâches comptables. L'autonomie de la batterie est très bonne, à 98 % après plusieurs jours d'utilisation. Le clavier est très léger, ce qui est appréciable lors de déplacement. Le seul inconvénient, le prix, comme pour tous les produits Apple.",1,sebastien M.,7 reviews,FR,5
1,Très très apprécié d’installer la…,"January 31, 2024",Très très apprécié d’installer la machine et reprendre l’ancienne et installation aussi de l’anti -calcaire et la personne très aimable et l’horaire respecté. Pas de frais de livraison.Je recommande.,1,Mme TURPIN Frede,3 reviews,FR,5
2,Machine à laver Bosch,"January 05, 2024","Commandée sur le site de Darty machine à laver Bosch, simplicité des programmes, bonne visibilité en façade, très silencieuse, c'est parfait pour le moment, c'est une excellente marque donc j'ai confiance.Livraison impeccable et très soignée, c'est parfait.",1,Christiane,56 reviews,FR,5
3,VENTE FORCEE ?,"November 08, 2023",Le 8 nov 2023 j'achète un ordinateur chez Darty Rambouillet. Le vendeur me dit : il n'y a pas le pack Office dans cet ordinateur.J'achète un pack office chez Darty Rambouillet.Il se trouve que mon pack office acheté quelques mois plus tôt fonctionne très bien sur mon nouvel ordi.Je demande le remboursement du 2e pack office à Darty.Aucune communication possible. Réponse catégorique : JE NE VOUS REMBOURSERAI PAS !,1,Annick Testu,5 reviews,FR,1
4,Fausse vente de crédit gratuit,"October 06, 2023",Fausse vente de crédit gratuitNon réponse du magasin de Blagnac à mon courrier.Vendeuse incompétente.L’incompétence du réseau Darty m’a couté 150€ et ils ont l’outrecuidance de me proposer un bon d’achat de 20€.Quelle belle entreprise.,1,Scherrer,3 reviews,FR,1


In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('OrdalieTech/Solon-embeddings-large-0.1', device='cuda')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.20k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [12]:
sentences = [str(x) for x in feedbacks_df['Title'] + feedbacks_df['Content']]

In [14]:

sentence_embeddings = model.encode(sentences, show_progress_bar=True, device='cuda')
sentence_embeddings.shape

Batches:   0%|          | 0/281 [00:00<?, ?it/s]

(8979, 1024)

In [28]:
#@Insight Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=15)
umap_embeds = reducer.fit_transform(sentence_embeddings)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'Title': feedbacks_df['Title'], "Content": feedbacks_df['Content']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]
df_explore


,Title,Content,x,y
0,Achat Magic Keyboard avec Touch ID et pave numerique chez Darty,"Toujours un très bon service de Darty. La récupération du produit a été rapide en magasin. Le personnel est très accueillant et sympathique.Achat d'un clavier Magic Keyboard avec Touch ID et pave numerique : le clavier est de très bonne qualité. La touche Touch ID permet de s'authentifier rapidement et le pavé numérique est très pratique lorsque l'on programme ou réalise des tâches comptables. L'autonomie de la batterie est très bonne, à 98 % après plusieurs jours d'utilisation. Le clavier est très léger, ce qui est appréciable lors de déplacement. Le seul inconvénient, le prix, comme pour tous les produits Apple.",9.893675,3.800933
1,Très très apprécié d’installer la…,Très très apprécié d’installer la machine et reprendre l’ancienne et installation aussi de l’anti -calcaire et la personne très aimable et l’horaire respecté. Pas de frais de livraison.Je recommande.,10.186836,2.809929
2,Machine à laver Bosch,"Commandée sur le site de Darty machine à laver Bosch, simplicité des programmes, bonne visibilité en façade, très silencieuse, c'est parfait pour le moment, c'est une excellente marque donc j'ai confiance.Livraison impeccable et très soignée, c'est parfait.",8.702872,2.367509
3,VENTE FORCEE ?,Le 8 nov 2023 j'achète un ordinateur chez Darty Rambouillet. Le vendeur me dit : il n'y a pas le pack Office dans cet ordinateur.J'achète un pack office chez Darty Rambouillet.Il se trouve que mon pack office acheté quelques mois plus tôt fonctionne très bien sur mon nouvel ordi.Je demande le remboursement du 2e pack office à Darty.Aucune communication possible. Réponse catégorique : JE NE VOUS REMBOURSERAI PAS !,4.861728,7.848199
4,Fausse vente de crédit gratuit,Fausse vente de crédit gratuitNon réponse du magasin de Blagnac à mon courrier.Vendeuse incompétente.L’incompétence du réseau Darty m’a couté 150€ et ils ont l’outrecuidance de me proposer un bon d’achat de 20€.Quelle belle entreprise.,5.767188,6.760652
...,...,...,...,...
8974,Globalement bien,"Bonne qualité des communications téléphoniques, trés bon débit internet, tarifs assez transparents, sans désagréables surprises.",11.600075,3.063365
8975,nouvel adherent,"je viens de souscrire aà l'offre darty internet et j'ai a plusieurds reprises appelé le service assistance qui à chaque fois à répondu tres aimablement et professionnellemnt efficacité rapidité , compétence à tous les problèmes que je rencontrais avec mon ordi , super , j n'a pas connu cela avec mes autres fournisseurs et en plus le prix est res attractif et concurrentiel , bravo Darty",9.601910,1.769595
8976,Mamac,service SAV LUXEMBOURG archi nul totalement incompétent au niveau informatique Ordi Acer sous garantie . J'ai dû payer des frais pour un formatage et une sauvegarde de mes données. Bref la garantie Darty c'est ZERO. N'achetez jamais un ordi chez Darty .,4.773075,8.049146
8977,Très bon,"Je suis toujours très satisfaite de ce site, rien à redire !",12.100242,3.194645


In [30]:
df_explore = df_explore[:5000]

In [31]:

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    #color='InsightParent',
    tooltip=['Title', 'Content']
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)

In [18]:
df_explore

,Title,x,y
0,Achat Magic Keyboard avec Touch ID et pave numerique chez Darty,11.785733,3.358894
1,Très très apprécié d’installer la…,11.878098,3.541754
2,Machine à laver Bosch,10.732310,2.992283
3,VENTE FORCEE ?,5.401579,7.329250
4,Fausse vente de crédit gratuit,6.754197,6.841990
...,...,...,...
8974,Globalement bien,13.074779,3.387192
8975,nouvel adherent,11.330773,2.163356
8976,Mamac,5.203953,7.474575
8977,Très bon,13.826571,3.795296


In [19]:
df_explore["Insight"] = 1

In [20]:
other_sentences = ["Les gens sont mécontents", "Les gens sont très contents", "Le produit est parfait", "Le produit doit être amélioré", "délais", "réactivité", "incohérence"]
other_sentence_embeddings = model.encode(other_sentences)

In [21]:
# Prepare the data to plot and interactive visualization
# using Altair


In [22]:
other_umap_embeds = reducer.transform(other_sentence_embeddings)

other_df = pd.DataFrame({"text":other_sentences, 'x': other_umap_embeds[:,0], 'y': other_umap_embeds[:,1], "Insight": 0})
other_df

,text,x,y,Insight
0,Les gens sont mécontents,8.639913,6.146481,0
1,Les gens sont très contents,13.931848,3.175931,0
2,Le produit est parfait,13.789123,3.885216,0
3,Le produit doit être amélioré,13.044131,3.604502,0
4,délais,12.368299,5.021390,0
5,réactivité,-7.099067,19.678085,0
6,incohérence,9.824716,6.770283,0


In [23]:
df_explore_new = pd.concat([df_explore, other_df])

In [24]:
# Plot
chart = alt.Chart(df_explore_new).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text'],
    color='Insight',
).properties(
    width=700,
    height=400
)
chart.interactive()

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.Chart(...)

In [25]:
df_explore_new.head()

,Title,x,y,Insight,text
0,Achat Magic Keyboard avec Touch ID et pave numerique chez Darty,11.785733,3.358894,1,NaN
1,Très très apprécié d’installer la…,11.878098,3.541754,1,NaN
2,Machine à laver Bosch,10.732310,2.992283,1,NaN
3,VENTE FORCEE ?,5.401579,7.329250,1,NaN
4,Fausse vente de crédit gratuit,6.754197,6.841990,1,NaN


In [26]:
df_explore_new['text'].to_list()

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [27]:
df_explore_new['embedding'] = reducer.transform(df_explore_new['text'])

ValueError: could not convert string to float: 'Les gens sont mécontents'

In [ ]:
umap_embeds